In [1]:
import numpy as np
from sklearn.metrics import jaccard_similarity_score

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/comoda_data.csv")

In [5]:
df = df.replace(-1.0, np.nan)
df = df.dropna(how="any")

## Clean Data Here

In [6]:
## CONSTANTS ASSUMPTION
sigma = 1
threshold = 0.1
columns = df.columns.tolist()
columns.remove("rating")
columns.append("rating")
df = df[columns]
X = df[columns[2:-1]]
y = df[columns[-1:]]
df = df.reset_index(drop=True)
#df

In [7]:
class neighbors:
    def __init__(self, X, y):
        self.model = KNeighborsClassifier(n_neighbors=10)
        self.model.fit(X,y)
    def get_neighbors_data(self, user, itemID):
        #all_neighbors_indices = self.model.kneighbors(user[2:-1])[1][0]
        #print(all_neighbors_indices)
        #neighbors_data = df.iloc[all_neighbors_indices]
        neighbors_data = df[(df.itemID == itemID)]
        #print(neighbors_data)
        self.good_neighbors = pd.DataFrame()
        #print(neighbors_data.shape)
        threshold = 0.3
        for index, row in neighbors_data.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], user[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.good_neighbors=self.good_neighbors.append(row, ignore_index=True)
                #print(row)
                #print(good_neighbors)
                #print("Inside Threshold")
            #print(self.good_neighbors)
        #print(self.good_neighbors)
        return self.good_neighbors

In [8]:
class similarity_class:
    def __init__(self):
        pass
    def jaccard_similarity(self, x, y):
        return jaccard_similarity_score(x,y)

In [9]:
class neighborhood_contribution:
    def __init__(self):
        pass
    def neighbor_rating(self, neighbor, itemID, sigma, threshold):
        self.ratings_sum=0
        self.similarity_sum=0
        #print(type(user))
        #print(user)
        #print(neighbor[2:10])
        #print(df.itemID)
        ratings = df[(df.userID == neighbor.userID) & (df.itemID == itemID)]
        #print(ratings.shape)
        for index, user_rating in ratings.iterrows():
            similarity = jaccard_similarity_score(neighbor[2:-1],user_rating[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.ratings_sum += user_rating.rating * similarity
                self.similarity_sum += similarity
                #print(self.similarity_sum)
            #print(self.ratings_sum)
            #print(neighbor[-10:],user_rating[-10:])
        #print("Rating sum")
        #print(self.ratings_sum)
        #print("Similarity Sum")
        #print(self.similarity_sum)
        try:
            rating = self.ratings_sum / self.similarity_sum
            #rating = ratings.rating.mean()
        except:
            rating = 0
        #print(rating)
        return rating
    def neighbor_average(self,neighbor,sigma,threshold):
        ratings_of_neighbor = df[df.userID == neighbor.userID]#.rating.mean()
        rating_sum=0;
        count=0;
        for index, row in ratings_of_neighbor.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], neighbor[2:-1])
            if similarity > threshold:
                rating_sum+=row.rating
                count+=1
        average_rating_in_given_context = rating_sum/count
        #print(average_rating_in_given_context)
        return average_rating_in_given_context
    def baseline_rating(self, itemID):
        average_item_rating = df[df.itemID == itemID].rating.mean()
        return average_item_rating

In [10]:
def numerator(neighborhood_contribution_rating, similarity_of_neighbors):
    numerator_sum=0
    #print(neighborhood_contribution_rating)
    #print(similarity_of_neighbors)
    for neighbor_rating, similarity in zip(neighborhood_contribution_rating,similarity_of_neighbors):
        numerator_sum+= neighbor_rating*similarity
    return numerator_sum

In [11]:
def predict_rating(user,itemID):
    similarity_function = similarity_class()
    user_neighbors = neighbors(X,y)
    user_neighborhood_contribution = neighborhood_contribution()
    neighbors_data = user_neighbors.get_neighbors_data(user,itemID)
    #print(neighbors_data.shape)
    #print(neighbors_data)
    #similarity_of_neighbors = [similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]) for neighbor in neighbors_data]
    similarity_of_neighbors = []
    neighborhood_contribution_rating = []
    #print(neighbors_data.shape)
    for index, neighbor in neighbors_data.iterrows():
        similarity_of_neighbors.append(similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]))
        neighborhood_contribution_rating.append(user_neighborhood_contribution.neighbor_rating(neighbor,itemID,1,0.1) - user_neighborhood_contribution.neighbor_average(neighbor,1,0.1))
#     neighborhood_contribution_rating = [user_neighborhood_contribution.neighbor_rating(neighbor,item) - user_neighborhood_contribution.neighbor_average(neighbor)
#                                         for neighbor in neighbors_data]
    numerator_value = numerator(neighborhood_contribution_rating, similarity_of_neighbors)
    baseline_rating = user_neighborhood_contribution.baseline_rating(itemID)
    #print(numerator_value)
    #print(similarity_of_neighbors)
    #print(baseline_rating)
    final_rating = baseline_rating + numerator_value/ sum(similarity_of_neighbors)
    return final_rating

In [12]:
# if __name__ == "main":
#     userId = 20
#     iteId = 30
#     rating = predict_rating(20,30)
#     print(rating)

In [13]:
predict_rating(df.iloc[37], 47)
#df.iloc[5]"

5.0479068684211841

In [14]:
#df[df.itemID == 47]

In [15]:
#df.iloc[37]

In [16]:
from sklearn.metrics import mean_squared_error

In [19]:
#y_true = [df.iloc[i].rating for i in range(700,800)]

In [20]:
#y_pred = [predict_rating(df.iloc[i], df.iloc[i].itemID) for i in range(700,800)]

In [ ]:
#y_pred

In [21]:
#mean_squared_error(y_true, y_pred)

## Pyswarm Implementation

In [23]:
# from pyswarm import pso

# def banana(x):
#     x1 = x[0]
#     x2 = x[1]
#     return x1**4 - 2*x2*x1**2 + x2**2 + x1**2 - 2*x1 + 5

# def con(x):
#     x1 = x[0]
#     x2 = x[1]
#     return [-(x1 + 0.25)**2 + 0.75*x2]

# lb = [-3, -1]
# ub = [2, 6]

# xopt, fopt = pso(banana, lb, ub, f_ieqcons=con)

In [24]:
from pyswarm import pso

In [32]:
len(df.rating.unique())

5

In [33]:
columns = df.columns

In [38]:
columns = columns[2:-1]

In [40]:
columns

Index(['city', 'country', 'time', 'daytype', 'season', 'location', 'weather',
       'social', 'endEmo', 'dominantEmo', 'mood', 'physical', 'decision',
       'interaction', 'director', 'movieCountry', 'movieLanguage', 'movieYear',
       'genre1', 'genre2', 'genre3', 'actor1', 'actor2', 'actor3', 'budget'],
      dtype='object')

In [58]:
from collections import defaultdict
size_of_feature = defaultdict(dict)
for column in columns:
    size_of_feature[column]['length'] = len(df[column].unique())
    size_of_feature[column]['names'] = df[column].unique()
    

In [61]:
#size_of_feature

In [51]:
size_of_feature['actor1']

329

In [68]:
df.actor1.max()

1994.0

In [69]:
df.actor1.describe()

count    1114.000000
mean     1012.370736
std       568.663979
min         4.000000
25%       531.000000
50%       969.000000
75%      1518.250000
max      1994.000000
Name: actor1, dtype: float64

In [84]:
feature_dict = defaultdict(dict)
for k,v in size_of_feature.items():
    for i,j in v.items():
        for name in v['names']:
            feature_dict[k][name] = sum(df[k] == name)
            #print(k,name)

In [92]:
#feature_dict

In [91]:
#df[df.columns[2:]].corr(method='kendall', min_periods=1)

In [ ]:
x = []